Find the compliance rate per day per participant
====

In [2]:
"""
Read in the smartwatch data

Throw away catchups since we don't care about them here

"""
from ema import clean

meal_info = clean.cleaned_smartwatch(keep_catchups=False)

/home/mh19137/seaco_malaysia/ema/clean.py:189: UserWarning: Long catchup: not marking Meal at 2022-05-19 09:02:49 as catchup
  warnings.warn(


 Open ended catchup at 2022-02-11 20:02:05 ended by No response at 2022-02-12 08:02:11
 Open ended catchup at 2022-02-14 20:26:30 ended by No response at 2022-02-15 08:02:16
 Open ended catchup at 2022-04-19 08:01:14 ended by No catch-up at 2022-04-19 08:01:17
 Open ended catchup at 2022-06-27 15:53:14 ended by Catch-up start at 2022-06-27 15:54:06
 Open ended catchup at 2021-12-31 22:11:10 ended by No catch-up at 2022-01-01 08:01:15
 Open ended catchup at 2022-05-24 07:25:36 ended by Catch-up start at 2022-05-24 07:26:00
 Open ended catchup at 2022-03-21 08:01:57 ended by No catch-up at 2022-03-21 08:02:04
 Open ended catchup at 2022-03-07 18:33:30 ended by No catch-up at 2022-03-07 18:33:34
 Open ended catchup at 2022-03-07 18:37:08 ended by No catch-up at 2022-03-07 18:37:14
 Open ended catchup at 2022-03-07 19:29:23 ended by No response at 2022-03-08 08:02:56
 Open ended catchup at 2022-03-08 08:30:40 ended by No catch-up at 2022-03-08 08:31:08
 Open ended catchup at 2022-03-08 09:

In [13]:
"""
Collapse entries into a bool

"""
meal_info["entry"] = meal_info["meal_type"].isin(
    {"Meal", "Drink", "Snack", "No food/drink"}
)

Datetime
2021-11-03 09:00:43     True
2021-11-03 09:00:49     True
2021-11-03 10:01:03     True
2021-11-03 11:01:07     True
2021-11-03 12:01:16     True
                       ...  
2022-07-26 16:12:14     True
2022-07-26 17:12:20     True
2022-07-26 18:14:20    False
2022-07-26 19:14:24     True
2022-07-26 20:14:34     True
Name: entry, Length: 7052, dtype: bool

In [11]:
"""
Group the entries by participant and time - two entries are marked as the same if they are by the same user and within 5 minutes of each other

"""
import numpy as np
import pandas as pd

collapsed_meal_info = pd.DataFrame()

meal_length = pd.Timedelta("5 minutes")
for p_id, group in meal_info.groupby("p_id"):
    assert group.index.is_monotonic_increasing

    # Mark which ones are near enough each other to be considered the same
    n_entries = len(group)
    keep = np.ones(n_entries, dtype=bool)

    for i in range(1, n_entries):
        if group.index[i] - group.index[i - 1] < meal_length:
            keep[i] = False

    # Append to the new dataframe
    collapsed_meal_info = pd.concat([collapsed_meal_info, group[keep]])

7052
6192


,x_id,meal_type,portion_size,utensil,location,week_day,p_id,ramadanstart,ramadanend,delta,catchup_category,catchup_flag,entry_in_ramadan,first_in_ramadan,last_in_ramadan,all_in_ramadan,any_in_ramadan
Datetime,,,,,,,,,,,,,,,,,
2022-04-09 08:02:16,306,No response,No response,No response,No response,Saturday,141,03apr2022,01may2022,1 days 08:02:16,NaN,False,True,True,True,True,True
2022-04-09 09:02:24,307,No food/drink,No response,No response,No response,Saturday,141,03apr2022,01may2022,1 days 09:02:24,NaN,False,True,True,True,True,True
2022-04-09 10:04:24,308,No response,No response,No response,No response,Saturday,141,03apr2022,01may2022,1 days 10:04:24,NaN,False,True,True,True,True,True
2022-04-09 11:04:28,309,No food/drink,No response,No response,No response,Saturday,141,03apr2022,01may2022,1 days 11:04:28,NaN,False,True,True,True,True,True
2022-04-09 12:04:32,310,No food/drink,No response,No response,No response,Saturday,141,03apr2022,01may2022,1 days 12:04:32,NaN,False,True,True,True,True,True


In [1]:
"""
Experiment with the length of time, seeing how the length of time affects the number of entries in our dataframe

"""


''